In [2]:
# importar bibliotecas

import PyPDF2
import re
import sys
sys.path.insert(0, 'C:\\Users\\MatheusPereira\\OneDrive - Pontte\\Área de Trabalho\\automacaoRegistroCCI\\Scripts')
from lerContrato import lerContrato